In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
import pandas as pd
import re
import json as js
from typing_extensions import TypedDict, Annotated
from typing import  Literal
from unsloth import FastLanguageModel
import torch
from unsloth.chat_templates import CHAT_TEMPLATES
from transformers import TextStreamer
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
!pip show unsloth

Name: unsloth
Version: 2025.5.8
Summary: 2-5X faster LLM finetuning
Home-page: http://www.unsloth.ai
Author: Unsloth AI team
Author-email: info@unsloth.ai
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: accelerate, bitsandbytes, datasets, diffusers, hf_transfer, huggingface_hub, numpy, packaging, peft, protobuf, psutil, sentencepiece, torch, torchvision, tqdm, transformers, triton, trl, tyro, unsloth_zoo, wheel, xformers
Required-by: 


In [4]:
!pip show torch

Name: torch
Version: 2.6.0+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, bitsandbytes, cut-cross-entropy, fastai, peft, sentence-transformers, timm, torchaudio, torchdata, torchvision, unsloth, unsloth_zoo, xformers


In [5]:
!nvidia-smi

Wed May 28 17:00:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P0             28W /   70W |     102MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
logs = pd.DataFrame(columns=["agent","prompt","response","document","budget_allocated","grantee_evaluation"])

In [7]:
user_message = """
Hello Supervisor,

I’d like your assistance in preparing a grant proposal for a new project I’m initiating. The project is focused on improving early childhood reading literacy in underserved communities through a hybrid approach that combines local storytelling traditions with structured reading support.

Low literacy rates in early primary education often lead to long-term academic challenges and reduced opportunities. My goal is to establish a community-based literacy development program that emphasizes:

Training early education teachers in phonics-based and comprehension-focused instruction,

Creating culturally relevant reading materials that reflect children’s environments and languages,

Establishing mobile reading corners and pop-up libraries,

Engaging parents through storytelling nights and home reading kits.

So far, I have:

Partnered with three local schools and two community libraries,

Collected preliminary reading assessment data from over 200 children aged 5–8,

Worked with local authors to begin drafting short illustrated stories in native dialects.

For this grant, I’d like funding for:

Publishing and distributing storybooks tailored to local linguistic and cultural contexts,

Organizing teacher training workshops and ongoing mentorship,

Deploying mobile library units to reach remote or underserved areas,

Producing family engagement materials and facilitating community storytelling events,

Hiring educational researchers to assess reading progress and inform iterative improvements.

Please begin by outlining the major proposal sections and suggesting any structural improvements. Also flag any challenges related to curriculum integration, linguistic diversity, or parental involvement that we should address early.

Best regards,
"""

In [8]:
class message(TypedDict):
    sender: str
    comments: str
    document: str
    budget_allocation: str
    grantee_evaluation: str
    user_requests: str

In [9]:
json_file = None

In [10]:
max_seq_length = 1000000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [11]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.5.8: Fast Qwen3 patching. Transformers: 4.52.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/Qwen3-8B-unsloth-bnb-4bit can only handle sequence lengths of at most 40960.
But with kaiokendev's RoPE scaling of 24.414, it can be magically be extended to 1000000!


model.safetensors.index.json:   0%|          | 0.00/144k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

In [12]:
FastLanguageModel.for_inference(model)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096, padding_idx=151654)
    (layers): ModuleList(
      (0-2): 3 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_

In [13]:
class document_formulator():
    def __init__(self, llm,rules):
        self.rules = {
            "role": "system",
            "content": rules
        }
        self.llm = llm
        self.memory = [self.rules]
    def __call__(self, user_message: str) -> message:
        self.memory.append({
            "role": "user",
            "content": "The supervisor commented that "+ user_message['comments']
        })
        self.memory.append({
            "role": "user",
            "content": "That is user request: "+ user_message['user_requests']
        })
        self.memory.append({
            "role": "user",
            "content": "Document created untill now: "+ user_message['document']
        })
        self.memory.append({
            "role": "user",
            "content": "Grantee evaluation until now: "+ user_message['grantee_evaluation']
        })
        self.memory.append({
            "role": "user",
            "content": "Budget allocated untill now: "+ user_message['budget_allocation']
        })

        text_streamer = TextStreamer(tokenizer)
        json_response = ""
        inputs = tokenizer.apply_chat_template(self.memory, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
        gen = model.generate(inputs, streamer = text_streamer,
                                    max_new_tokens=4096,
                                    temperature=0.8,
                                    top_p=0.95)

        compleation = tokenizer.decode(gen[0])
        start_res = compleation.rfind("<|im_start|>assistant")
        prompt = compleation[:start_res]
        response = compleation[start_res:]
        json_response = response.split("<think>")[1].split("</think>")[1][:-len("<|im_end|>")].strip()
        json_parsed = js.loads(json_response)


        global logs
        logs.loc[len(logs)] = {
            "agent": "document_formulator",
            "prompt": prompt,
            "response": response,
            "document": str(js.dumps(json_parsed['document'])),
            "budget_allocated": user_message['budget_allocation'],
            "grantee_evaluation": user_message['grantee_evaluation']
        }

        self.memory = [self.rules]
        self.memory.append({
            "role": "user",
            "content": "last comment the supervisor gaved you: "+ user_message['comments']
        })
        self.memory.append({
            "role": "user",
            "content": "your last comments to the supervisor : "+ json_parsed['comments']
        })

        return message(
            sender="document_formulator",
            comments=json_parsed['comments'],
            document= str(js.dumps(json_parsed['document'])),
            budget_allocation=user_message['budget_allocation'],
            grantee_evaluation=user_message['grantee_evaluation'],
            user_requests=user_message['user_requests']
        )

In [14]:
document_formulator_rules = """
You are a document formulator agent
your job is to follow the rules of the supervisor agent and create a document based on the user request and the task description you received from the supervisor
and grantee simulator evaluation to your work.
You have multiple rules you must follow.
1. You must follow the rules of the supervisor agent and create a document based on the user request and the task description you received from the supervisor.
2. You must not do any other tasks that are not related to document formulation such as budget allocation
3. if anything related to budget allocation is not given you can fill it blank untill you get budget allocation plan.
for example :
  For total budget: $[to be filled]
  not
  For total budget: $100,000
 but if the budget is already allocated with numbers you could put numbers normally in the document not placeholders.
4. reply with only the JSON response that contains the document you created and if there are any updates you just send the full version with your updates in the following format:
"
{
    "document": [document in json format],
    "comments": "[your comments about the document you created to the supervisor]"
}
"

for example:

"
{
    "document": {
        "title": "Project Scope and Methodology",
        "sections": [
            {
                "section_title": "Introduction",
                "content": "This document outlines the project scope and methodology for the AI-based detection of misinformation in short-form videos."
            },
            {
                "section_title": "Project Scope",
                "content": "The project aims to develop a system that detects misinformation in short-form videos on platforms like TikTok, Instagram Reels, and YouTube Shorts."
            },
            {
                "section_title": "Methodology",
                "content": "The methodology includes audio transcription, semantic analysis, video content recognition, and cross-referencing claims with trusted knowledge bases."
            }
        ]
    },
    "comments": "The document outlines the project scope and methodology based on the user request."
}
"

5. Do not include any additional text or explanations in your response, just the JSON response mentioned in rule 3.
6. Remember that you must give comments to the supervisor about the document you created and how it relates to the user request in the required formate as rule 3.
"""
document_formulator_agent = document_formulator(llm=model, rules=document_formulator_rules)

In [15]:
class budget_allocator_agent():
    def __init__(self, model, rules):
        self.rules = {
            "role": "system",
            "content": rules
        }
        self.model = model
        self.memory = [self.rules]
    def __call__(self, user_message: str) -> message:
        self.memory.append({
            "role": "user",
            "content": "The supervisor commented that "+ user_message['comments']
        })
        self.memory.append({
            "role": "user",
            "content": "Document created untill now: "+ user_message['document']
        })
        self.memory.append({
            "role": "user",
            "content": "Grantee evaluation until now: "+ user_message['grantee_evaluation']
        })
        self.memory.append({
            "role": "user",
            "content": "The user request is: "+ user_message['user_requests']
        })
        self.memory.append({
            "role":"user",
            "content":"budget you allocated untill now"+ user_message['budget_allocation']
        })



        text_streamer = TextStreamer(tokenizer)
        json_response = ""
        inputs = tokenizer.apply_chat_template(self.memory, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
        gen = model.generate(inputs, streamer = text_streamer,
                                    max_new_tokens=131072,
                                    temperature=0.8,
                                    top_p=0.95)

        compleation = tokenizer.decode(gen[0])
        start_res = compleation.rfind("<|im_start|>assistant")
        prompt = compleation[:start_res]
        response = compleation[start_res:]
        json_response = response.split("<think>")[1].split("</think>")[1][:-len("<|im_end|>")].strip()
        json_parsed = js.loads(json_response)
        global logs
        logs.loc[len(logs)] = {
            "agent": "budget_allocator",
            "prompt": prompt,
            "response": response,
            "document": user_message['document'],
            "budget_allocated": str(js.dumps(json_parsed['budget_allocation'])),
            "grantee_evaluation": user_message['grantee_evaluation']
        }

        self.memory = [self.rules]
        self.memory.append({
            "role": "user",
            "content": "last comment the supervisor gaved you: "+ user_message['comments']
        })
        self.memory.append({
            "role": "user",
            "content": "your last comments to the supervisor : "+ json_parsed['comments']
        })

        return message(
            sender="budget_allocator",
            comments=json_parsed['comments'],
            document=user_message['document'],
            budget_allocation=str(js.dumps(json_parsed['budget_allocation'])),
            grantee_evaluation=user_message['grantee_evaluation'],
            user_requests=user_message['user_requests']
        )

In [16]:
budget_allocator_agent_rules = """
You are a budget allocator agent
your job is to follow the rules of the supervisor agent and create a budget based on the user request and the task description you received from the supervisor
and grantee simulator evaluation to your work.
You have multiple rules you must follow.
1. You must follow the rules of the supervisor agent and create a budget based on the user request and the task description you received from the supervisor.
2. reply with only the JSON response that contains the budget you created in the following format:
"
{
    "budget_allocation": [budget in json format],
    "comments": "[your comments about the budget you created to the supervisor]"
}
"
for example:
{
    "budget_allocation": {
        "total_budget": 50000,
        "breakdown": {
            "dataset_expansion": 20000,
            "annotators": 15000,
            "media_analysis_tools": 10000,
            "cloud_services": 5000
        }
    },
    "comments": "The budget allocation is based on the user request and the task description provided by the supervisor."
}
3. Do not include any additional text or explanations in your response, just the JSON response mentioned in rule 3.
4. Remember that you must give comments to the supervisor about the budget you created and how it relates to the user request in the required format as rule 3.
"""
budget_allocator_agent = budget_allocator_agent(model=model, rules=budget_allocator_agent_rules)

In [17]:
class grantee_simulator_agent():
    def __init__(self, llm,rules):
        self.llm = llm
        self.rules = {
            "role": "system",
            "content": rules
        }
        self.memory = [self.rules]
    def __call__(self, user_message: str) -> message:
        self.memory.append({
            "role": "user",
            "content": "The supervisor commented that "+ user_message['comments']
        })
        self.memory.append({
            "role": "user",
            "content": "The user request : "+ user_message['user_requests']
        })
        self.memory.append({
            "role": "user",
            "content": "Documents created untill now: "+ user_message['document']
        })
        self.memory.append({
            "role": "user",
            "content": "Your evaluation until now: "+ user_message['grantee_evaluation']
        })
        self.memory.append({
            "role": "user",
            "content": "Budget allocated untill now: "+ user_message['budget_allocation']
        })

        text_streamer = TextStreamer(tokenizer)
        json_response = ""
        inputs = tokenizer.apply_chat_template(self.memory, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
        gen = model.generate(inputs, streamer = text_streamer,
                                    max_new_tokens=4096,
                                    temperature=0.8,
                                    top_p=0.95)

        compleation = tokenizer.decode(gen[0])
        start_res = compleation.rfind("<|im_start|>assistant")
        prompt = compleation[:start_res]
        response = compleation[start_res:]
        json_response = response.split("<think>")[1].split("</think>")[1][:-len("<|im_end|>")].strip()
        json_parsed = js.loads(json_response)

        global logs
        logs.loc[len(logs)] = {
            "agent": "grantee_simulator",
            "prompt": prompt,
            "response": response,
            "document": user_message['document'],
            "budget_allocated": user_message['budget_allocation'],
            "grantee_evaluation": str(js.dumps(json_parsed['grantee_evaluation'])),
        }

        self.memory = [self.rules]
        self.memory.append({
            "role": "user",
            "content": "last comment the supervisor gaved you: "+ user_message['comments']
        })
        self.memory.append({
            "role": "user",
            "content": "your last comments to the supervisor : "+ json_parsed['comments']
        })

        return message(
            sender="grantee_simulator",
            comments=json_parsed['comments'],
            document=user_message['document'],
            budget_allocation= user_message['budget_allocation'],
            grantee_evaluation=str(js.dumps(json_parsed['grantee_evaluation'])),
            user_requests=user_message['user_requests']
        )

In [18]:
grantee_simulator_agent_rules = """
You are a grantee simulator agent
your job is to follow the rules of the supervisor agent and create a grantee evaluation based on the user request and the task description you received from the supervisor
and document formulator and budget allocator agent evaluation to your work.
You have multiple rules you must follow.
1. You must follow the rules of the supervisor agent and create a grantee evaluation based on the user request and the task description you received from the supervisor.
2. You must not do any other tasks that are not related to grantee evaluation such as document formulation or budget allocation if you have no numbers just but variables.
3. reply with only the JSON response that contains the grantee evaluation you created in the following format as checklist that you made:
{
    "grantee_evaluation": {
        "evaluation": [
            {
                "criteria": "Project Scope",
                "status": "satisfactory",
                "comments": "[your comments on this]"
            },
            {
                "criteria": "Methodology",
                "status": "needs improvement",
                "comments": "[yourn comments on this]"
            },
            .....
        ]
    },
    "comments": "[your comments about the grantee evaluation you created to the supervisor]"
}
4. consider to update the checklist if you got new information with the required updates you give to the supervisor.
5. Do not include any additional text or explanations in your response, just the JSON response mentioned in rule 3.
6. Remember that you must give comments to the supervisor about the grantee evaluation you created and how it relates to the user request in the required format as rule 3.
7. If you are fully satisfied for all aspects of the work done and nothing need any improvements or clarification for the work done you would comment "FINISH" to the supervisor
    example:
    {
        "grantee_evaluation": {
            "evaluation": [
                {
                    "criteria": "Project Scope",
                    "status": "satisfactory",
                    "comments": "The project scope is well-defined and aligns with the user request."
                },
                .....
            ]
        },
        "comments": "FINISH"
    }
    if not you would give comments to the supervisor:
    example:
    {
        "grantee_evaluation": {
            "evaluation": [
                {
                    "criteria": "Project Scope",
                    "status": "satisfactory",
                    "comments": "The project scope is well-defined and aligns with the user request."
                },
                {
                    "criteria": "Methodology",
                    "status": "needs improvement",
                    "comments": "The methodology needs more detail on how the AI models will be implemented."
                },
                .....
            ]
        },
        "comments": "you need to consider ..."
    }
"""

In [19]:
grantee_simulator_agent = grantee_simulator_agent(
    llm=model,
    rules=grantee_simulator_agent_rules
)

In [20]:
g = ""

In [22]:
class Supervisor_agent():
    def __init__(self, llm, tokenizer,summarizer_llm,rules):
        self.llm = llm
        self.rules = {
            "role": "system",
            "content": rules
        }
        self.memory = [self.rules]
        self.summarizer_llm = summarizer_llm
    def act(self, msg: message):
        sender = msg["sender"].strip()
        comments = msg["comments"].strip()
        if sender != "supervisor":
            self.memory.append(
                {
                    "role": "user",
                    "content": "The "+sender+" commented that "+comments
                }
            )
        document = msg["document"]
        budget_allocation = msg["budget_allocation"].strip()
        grantee_evaluation = msg["grantee_evaluation"].strip()
        user_message = msg["user_requests"].strip()


        self.memory.append(
            {
                "role": "user",
                "content": "\n User prompt: "+sender + " said to you that " + comments
            }
        )
        self.memory.append(
            {
                "role": "user",
                "content": "Documents created untill now: " + document
            }
        )
        self.memory.append(
            {
                "role": "user",
                "content": "Budget allocation: " + budget_allocation
            }
        )
        self.memory.append(
            {
                "role": "user",
                "content": "Grantee evaluation: " + grantee_evaluation
            }
        )
        self.memory.append(
            {
                "role": "user",
                "content": "User request: " + user_message
            }
        )
        text_streamer = TextStreamer(tokenizer)
        json_response = ""
        inputs = tokenizer.apply_chat_template(self.memory, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
        gen = model.generate(inputs, streamer = text_streamer,
                                    max_new_tokens=4096,
                                    temperature=0.8,
                                    top_p=0.95)

        compleation = tokenizer.decode(gen[0])
        start_res = compleation.rfind("<|im_start|>assistant")
        prompt = compleation[:start_res]
        response = compleation[start_res:]
        json_response = response.split("<think>")[1].split("</think>")[1][:-len("<|im_end|>")].strip()
        j = js.loads(json_response)


        global logs
        logs.loc[len(logs)] = {
            "agent": "supervisor",
            "prompt": prompt,
            "response": response,
            "document": document,
            "budget_allocated": budget_allocation,
            "grantee_evaluation": grantee_evaluation
        }

        messages = []
        flag = False
        for i in j:
            messages.append(f"The Supervisor commented to the {i['send_to']} : " + i['comments'] + "\n")
            if i['send_to'] == 'user':
                self.memory.append({
                    "role": "user",
                    "content": "You have now send your task to the user"
                })
            elif i['send_to'] == 'document_formulator':
                self.memory.append(
                    {
                        "role": "user",
                        "content": "You have now send your task to the document formulator"
                    }
                )
                supervisor_msg = message(
                    sender='Supervisor',
                    comments=i['comments'],
                    document=document,
                    budget_allocation=budget_allocation,
                    grantee_evaluation=grantee_evaluation,
                    user_requests="The user request: " + user_message
                )
                msg = document_formulator_agent(supervisor_msg)
                self.memory.append(
                    {
                        "role":"user",
                        "content":"Document by the document_formulator: " + msg['document']
                    }
                )

                self.memory.append(
                    {
                        "role":"user",
                        "content":"The Document formulator commented that : " + msg['comments']
                    }
                )
                document = msg['document']

                messages.append("The document_formulator commented that to the supervisor : " + msg['comments'] + "\n")
            elif i['send_to'] == 'budget_allocator':
                self.memory.append({
                    "role": "user",
                    "content": "You have now send your task to the budget allocator"
                })
                budget_allocator_msg = message(
                    sender='supervisor',
                    comments=i['comments'],
                    document=document,
                    budget_allocation=budget_allocation,
                    grantee_evaluation=grantee_evaluation,
                    user_requests="The user request: " + user_message
                )
                msg = budget_allocator_agent(budget_allocator_msg)
                self.memory.append({
                    "role":"user",
                    "content":"Now the budget allocator created the following budget: " + msg['budget_allocation']
                })
                self.memory.append({
                    "role":"user",
                    "content":"The Budget allocator commented that : " + msg['comments']
                })
                budget_allocation = msg['budget_allocation']

                messages.append("The budget_allocator commented that to the supervisor : " + msg['comments'] + "\n")
            elif i['send_to'] == 'grantee_simulator':
                self.memory.append(
                    {
                        "role": "user",
                        "content": "You have now send your task to the grantee simulator"
                    }
                )
                self.memory.append(
                    {
                        "role": "user",
                        "content": "Now the grantee simulator will evaluate your proposal based on the user request and the task description you received from the supervisor"
                    }
                )
                grantee_simulator_msg = message(
                    sender='Supervisor',
                    comments=i['comments'],
                    document=document,
                    budget_allocation=budget_allocation,
                    grantee_evaluation=grantee_evaluation,
                    user_requests="The user request: " + user_message
                )
                msg = grantee_simulator_agent(grantee_simulator_msg)
                self.memory.append({
                    "role":"user",
                    "content":"Now the grantee simulator created the following checklist: " + msg['grantee_evaluation']
                })
                self.memory.append({
                    "role":"user",
                    "content":"The Grantee simulator commented that : " + msg['comments']
                })
                grantee_evaluation = msg['grantee_evaluation']

                messages.append("The grantee_simulator commented that to the supervisor : " + msg['comments'] + "\n")

                flag == True
                if msg['comments'] =="FINISH":
                  return message(
                      sender="supervisor",
                      comments="The grantee_simulator is satisfied with the all work done",
                      document=document,
                      budget_allocation=budget_allocation,
                      grantee_evaluation=grantee_evaluation,
                      user_requests=user_message
                  )
            else:
                raise ValueError(f"Unknown worker: {i['send_to']}")

        if flag == False:
          self.memory.append(
              {
                  "role": "user",
                  "content": "You have now send to the grantee simulator to review the work done"
              })
          self.memory.append(
            {
                "role": "user",
                "content": "Now the grantee simulator will evaluate your proposal based on the user request and the task description you received from the supervisor"
            })
          grantee_simulator_msg = message(
              sender='Supervisor',
              comments="please evaluate the work done untill now",
              document=document,
              budget_allocation=budget_allocation,
              grantee_evaluation=grantee_evaluation,
              user_requests="This is user request: " + user_message)
          msg = grantee_simulator_agent(grantee_simulator_msg)
          self.memory.append({
              "role":"user",
              "content":"Now the grantee simulator created the following checklist: " + msg['grantee_evaluation']
              })
          self.memory.append({
              "role":"user",
              "content":"The Grantee simulator commented that : " + msg['comments']
              })
          grantee_evaluation = msg['grantee_evaluation']
          messages.append("The grantee_simulator commented that to the supervisor : " + msg['comments'] + "\n")
          if msg['comments'] =="FINISH":
            return message(
                sender="supervisor",
                comments="The grantee_simulator is satisfied with the all work done",
                document=document,
                budget_allocation=budget_allocation,
                grantee_evaluation=grantee_evaluation,
                user_requests=user_message)

        # Generate summary to the conversation
        messages = "".join(messages)
        summary = [
            {
                "role": "system",
                "content": "You are responsible for summarizing "
            }
        ]
        summary.append(
            {
                "role": "user",
                "content": "last conversations : " + messages
            }
        )
        summary = tokenizer.apply_chat_template(summary, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
        summary_gen = model.generate(summary)

        global g
        g = summary_gen
        summary_compleation = tokenizer.decode(summary_gen[0])
        start_res = summary_compleation.rfind("<|im_start|>assistant")
        prompt = summary_compleation[:start_res]
        response = summary_compleation[start_res:]
        # response without thinking
        response = response.split("<think>")[1].split("</think>")[1][:-len("<|im_end|>")].strip()
        self.memory= [self.rules]
        self.memory.append(
            {
                "role": "user",
                "content": response
            }
        )


        return message(
            sender="supervisor",
            comments=response,
            document=document,
            budget_allocation=budget_allocation,
            grantee_evaluation=grantee_evaluation,
            user_requests=user_message
        )


In [23]:
rules = """
        You are the supervisor your job is to manage and orchestrated between your workers who are document_formulator, budget_allocator, and grantee_simulator and give there orders to do.
        You have multiple rules you must follow.
        1. Interpret user requests or grantee messages and decide which worker is best suited:
            - document_formulator: create and updates the document based on the user request and the task description you received from the supervisor.
            - budget_allocator: create a budget based on the user request and the task description you received from the supervisor.
            - grantee_simulator: simulate the grantee based on the user request and the task description you received from the supervisor.
        2. When you got evaluation from the grantee_simulator you must consider it and assign what worker you will send task to
        3. Never do a job of any worker yourself. Always delegate to the appropriate worker as only allowed is to give instructions to the workers. You are not allowed to do any other tasks.
        4. You must not give order to worker that is not related to its duties that is described in the first rule.
        5. You are only allowed to respond with your comments "comments" to the worker and any of the workers you are talking to "send_to" without any introductions, additional text or explanations.
        in the following json format as response:
        [
            {
                "send_to": "[worker_name]",
                "comments": "[your instructions to that worker]"
            }
        ]
        6. If you want to do multiple orders you would follow this order:
        [
            {
                "send_to": "[worker_name]",
                "comments": "[your instructions to that worker]"
            },
            {
                "send_to": "[worker_name]",
                "comments": "[your instructions to that worker]"
            }
        ]
        """
supervisor = Supervisor_agent(llm=model,summarizer_llm=model, tokenizer=tokenizer,rules=rules)

In [24]:
msg = message(sender='user',comments=user_message,document="No document created yet",
            budget_allocation="No budget allocated yet", grantee_evaluation="No evaluation made yet", user_requests=user_message)
last_msg = msg

In [25]:
logs

,agent,prompt,response,document,budget_allocated,grantee_evaluation


In [26]:
last_msg

{'sender': 'user',
 'comments': '\nHello Supervisor,\n\nI’d like your assistance in preparing a grant proposal for a new project I’m initiating. The project is focused on improving early childhood reading literacy in underserved communities through a hybrid approach that combines local storytelling traditions with structured reading support.\n\nLow literacy rates in early primary education often lead to long-term academic challenges and reduced opportunities. My goal is to establish a community-based literacy development program that emphasizes:\n\nTraining early education teachers in phonics-based and comprehension-focused instruction,\n\nCreating culturally relevant reading materials that reflect children’s environments and languages,\n\nEstablishing mobile reading corners and pop-up libraries,\n\nEngaging parents through storytelling nights and home reading kits.\n\nSo far, I have:\n\nPartnered with three local schools and two community libraries,\n\nCollected preliminary reading as

In [27]:
.# for i in range(10):
while range(20):
  msg = supervisor.act(last_msg)
  last_msg = msg
  if last_msg['comments'] == "The grantee_simulator is satisfied with the all work done":
    print("-------------------------")
    print("-------------------------")
    print("-------------------------")
    break
  print("-------------------------")
  print("-------------------------")
  print("-------------------------")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Streaming output truncated to the last 5000 lines.
First, the methodology section needs more granular lesson plans and timelines. The current document mentions integrating storytelling with phonics but lacks specific examples. I should add detailed lesson plans, like how a folk tale is broken down into phonics exercises. Also, including a timeline with phases and specific activities would help. Maybe create a sample lesson plan and outline the implementation phases with examples of adapting folktales.

For curriculum alignment, the existing table has state standards but needs more detail. The supervisor wants teacher resources for seamless integration. I can expand the table to include more standards and add a section with resources like lesson templates or guides. Maybe include a sample lesson plan that aligns with specific standards.

Linguistic diversity strategies are too vague. The current section mentions dialect standardization but doesn't outline the protocols. I need to add sp

KeyboardInterrupt: 

In [28]:
logs.to_csv("logs9.csv")